In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

In [ ]:
data_list = ['BUS_STATION_BOARDING_MONTH_201901.csv',
             'BUS_STATION_BOARDING_MONTH_201902.csv',
             'BUS_STATION_BOARDING_MONTH_201903.csv',
             'BUS_STATION_BOARDING_MONTH_201904.csv',
             'BUS_STATION_BOARDING_MONTH_201905.csv',
             'BUS_STATION_BOARDING_MONTH_201906.csv',
             'BUS_STATION_BOARDING_MONTH_201907.csv',
             'BUS_STATION_BOARDING_MONTH_201908.csv',
             'BUS_STATION_BOARDING_MONTH_201909.csv',
             'BUS_STATION_BOARDING_MONTH_201910.csv',
             'BUS_STATION_BOARDING_MONTH_201911.csv',
             'BUS_STATION_BOARDING_MONTH_201912.csv']  #데이터 불러오기
bus_station = pd.read_csv('/content/drive/MyDrive/Datacampus_python/프로젝트/데이터/2019년버스정류소위치_최종.csv', encoding='utf-8')

df_final_list = [] # 마지막 데이터 병합을 위해 빈 리스트 생성

for data in data_list:  # 데이터 불러오고 정제과정 for문으로 단순화
  df = pd.read_csv(f'/content/drive/MyDrive/Datacampus_python/프로젝트/데이터/{data}', encoding='cp949') # 데이터 불러오기
  day = df['사용일자'].nunique() # 일평균을 위한 월별 일 수 구하기
  df = df.drop(['노선번호','노선명','등록일자'], axis=1) # 필요없눈 컬럼 정리
  df['총승하차승객수'] = df['승차총승객수'] + df['하차총승객수'] #총 승하차승객수 구하기
  df.loc[df['역명'] == '남산서울타워(가상)', '버스정류장ARS번호'] = '03320' # 가상버스 정류소에서 현재 실 버스 정류소로 바뀐 정류소 변환
  df.drop(df[(df['버스정류장ARS번호'] == '~')].index, inplace = True) # 결측값 처리(가상 버스 정류장)
  df = df.astype({'버스정류장ARS번호':int}) # 데이터 타입 변경
  df_left_JOIN = pd.merge(df, bus_station, left_on='버스정류장ARS번호', right_on='ARSID', how='left') #버스정류소 위치 지오코딩 데이터와 유동인구 데이터 조인
  df_left_JOIN = df_left_JOIN.dropna()


  df_left_JOIN= df_left_JOIN.dropna() #서울시 지역 외의 버스정류장 전처리
  df_final = pd.DataFrame(df_left_JOIN.groupby('행정구역_요약')['총승하차승객수'].sum()) # 일별 총승하차승객수 #버스정류소 총 갯수
  df_final['일일평균'] = round(df_final['총승하차승객수']/day) #월단위 일일평균 승하차승객수
  df_final_list.append(df_final) #월단위 데이터 통합을 위해 리스트에 추가

combined_df = pd.concat(df_final_list, axis = 1)
# 월별평균 승하차 승객수 데이터 전처리
month_list = [data.split('_')[4].split('.')[0][4:6] for data in data_list]
columns = [f'{month}_{metric}' for month in month_list for metric in ["총승하차승객수", "일일평균"]]
combined_df.columns = columns

# 연 승하차승객수 데이터 전처리
combined_df['연총승하차승객수'] = combined_df[[f'{month}_총승하차승객수' for month in month_list]].sum(axis=1)
combined_df['월평균 총승하차승객수'] = round(combined_df['연총승하차승객수']/12)

combined_df['월별일일평균승하차승객수'] = combined_df[[f'{month}_일일평균' for month in month_list]].sum(axis=1)
combined_df['연일일평균총승하차승객수'] = round(combined_df['월별일일평균승하차승객수']/12)

# 주소 자치구 행정동 데이터 정리
combined_df.reset_index(inplace=True)
temp_df = combined_df['행정구역_요약'].str.split(",", expand=True).rename(columns={0:"행정동", 1:"자치구"})
combined_df = pd.concat([combined_df, temp_df], axis=1).drop('행정구역_요약', axis=1)

print(combined_df)

In [ ]:
print(combined_df[combined_df['자치구'].isna()]) # 자치구 미입력 데이터 전처리
combined_df.loc[combined_df['행정동'] == '성산2동', '자치구'] = '마포구'
combined_df.loc[combined_df['행정동'] == '흑석동', '자치구'] = '동작구'
print(combined_df[combined_df['자치구'].isna()])

In [ ]:
final_df = combined_df[['자치구','행정동','월평균 총승하차승객수','월별일일평균승하차승객수','연일일평균총승하차승객수']] # 사용 컬럼 불러오기
final_df.tail()
# 주소 맵핑을 위해 공백제거
final_df['자치구'] = final_df['자치구'].str.strip()
final_df['행정동'] = final_df['행정동'].str.strip()
print(final_df)
final_df.to_csv('./2019년행정동별버스유동인구_수정전.csv', index=False, encoding='utf-8-sig')


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Datacampus_python/프로젝트/데이터/2019년행정동별버스유동인구_수정전.csv')

# 각 컬럼 데이터의 양쪽 공백 제거
data['자치구'] = data['자치구'].str.strip()
data['행정동'] = data['행정동'].str.strip()

# 자치구와 행정동을 합치는 함수
def combine_names(row):
    return row['자치구']+',' + row['행정동']

# 자치구와 행정동을 합친 새로운 열 생성
data['자치구_행정동'] = data.apply(combine_names, axis=1)

# 같은 자치구와 행정동을 가진 행들을 합치기
aggregated_data = data.groupby('자치구_행정동').agg({
    '월평균 총승하차승객수': 'sum',
    '월별일일평균승하차승객수': 'sum',
    '연일일평균총승하차승객수': 'sum'
}).reset_index()
aggregated_data.reset_index(inplace=True)
temp_df = aggregated_data['자치구_행정동'].str.split(",", expand=True).rename(columns={0:"자치구", 1:"행정동"})
aggregated_data = pd.concat([aggregated_data, temp_df], axis=1).drop('자치구_행정동', axis=1)

#print(aggregated_data)

final_df = aggregated_data[['자치구','행정동','월평균 총승하차승객수','월별일일평균승하차승객수','연일일평균총승하차승객수']]
final_df[final_df['자치구'].str.contains('구', na = False) == False] # 서울 자치구가 아닌 경기도 지역 삭제
filtered_df = final_df[final_df['자치구'].str.contains('구')]
filtered_df = filtered_df[filtered_df.자치구 != '구리시'] #서울 지역 자치구만 추출
print(filtered_df)
grouped = filtered_df.groupby('자치구')['행정동'].nunique()
print(grouped)


In [ ]:
filtered_df.to_csv('./2019년행정동별버스유동인구_최종본.csv', index=False, encoding='utf-8-sig')